---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
df_all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
df_all_homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [4]:
df_university = pd.read_table('university_towns.txt', names=['field'])
df_university.head()

,field
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [5]:
df_gdp = pd.read_excel('gdplev.xls')
df_gdp.head(10)

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2016-09-29 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN


In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df_university = pd.read_table('university_towns.txt', names=['Field'])
    df_university['is_state'] = df_university['Field'].str.contains(r'\[edit\]')
    df_university['Group'] = df_university['is_state'].cumsum()
    df_university['State'] = df_university.groupby('Group')['Field'].transform('first')
    df_university['State'] = df_university['State'].str.replace(r'\[edit\]', '')
    df_university['Field']  = df_university['Field'].str.replace(r' \(.+$', '')
    df_university = df_university.loc[~df_university['is_state']]
    
    df_university.rename(columns={'Field': 'RegionName'}, inplace=True)
    
    return df_university[['RegionName', 'State']]

get_list_of_university_towns().head(5)

,RegionName,State
1,Auburn,Alabama
2,Florence,Alabama
3,Jacksonville,Alabama
4,Livingston,Alabama
5,Montevallo,Alabama


In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=7).iloc[:, [4, 6]]
    df_gdp.columns = ['Quarter', 'GDP (billions)']
    df_gdp = df_gdp[df_gdp['Quarter'] >= '2000q1']
    
    for i in range(len(df_gdp) - 2):
        if (df_gdp.iloc[i][1] > df_gdp.iloc[i+1][1]) & (df_gdp.iloc[i+1][1] > df_gdp.iloc[i+2][1]):
            return df_gdp.iloc[i+1][0]

get_recession_start()

'2008q3'

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=7).iloc[:, [4, 6]]
    df_gdp.columns = ['Quarter', 'GDP (billions)']
    df_gdp = df_gdp[df_gdp['Quarter'] >= '2000q1']
    
    i = get_recession_start()
    i = df_gdp[df_gdp['Quarter'] == i].index.tolist()[0]
    
    df_gdp = df_gdp.loc[i:]
    for i in range(len(df_gdp)-2):
        if (df_gdp.iloc[i+2][1] > df_gdp.iloc[i+1][1]) & (df_gdp.iloc[i+1][1] > df_gdp.iloc[i][1]):
            return df_gdp.iloc[i+2][0]

get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df_gdp = pd.read_excel('gdplev.xls', skiprows=7).iloc[:, [4, 6]]
    df_gdp.columns = ['Quarter', 'GDP (billions)']
    df_gdp = df_gdp[df_gdp['Quarter'] >= '2000q1']
    
    bgn = get_recession_start()
    bgn = df_gdp[df_gdp['Quarter'] == bgn].index.tolist()[0]
    end = get_recession_end()
    end = df_gdp[df_gdp['Quarter'] == end].index.tolist()[0]
    
    df_gdp = df_gdp.loc[bgn:end+1]
    
    return df_gdp[df_gdp['GDP (billions)'] == df_gdp['GDP (billions)'].min()]['Quarter'].values[0]

get_recession_bottom()

'2009q2'

In [10]:
def new_col_names():
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]


def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_all_homes.drop(['Metro','CountyName','RegionID','SizeRank'], axis=1, inplace=1)
    df_all_homes['State'] = df_all_homes['State'].map(states)
    df_all_homes.set_index(['State','RegionName'], inplace=True)
    col = list(df_all_homes.columns)
    col = col[0:45]
    df_all_homes.drop(col, axis=1, inplace=1)
    
    df_all_homes = df_all_homes.groupby(pd.PeriodIndex(df_all_homes.columns, freq='Q'), axis=1).mean()
    df_all_homes.columns = new_col_names()
    
    return df_all_homes

convert_housing_data_to_quarters().head()

ValueError: For argument "inplace" expected type bool, received type int.

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df_all_homes  = convert_housing_data_to_quarters()
    
    recession_bgn = get_recession_start()
    index = df_all_homes.columns.get_loc(recession_bgn)-1
    bfr_recession = df_all_homes.iloc[:, index]
    recession_btt = get_recession_bottom()
    df_university = get_list_of_university_towns()
    
    prices_btt = df_all_homes.loc[:, recession_btt]
    df_all_homes['Ratio'] = 0.0
    df_all_homes['Ratio'] = bfr_recession.values/prices_btt.values
    
    # price_ratio=quarter_before_recession/recession_bottom
    df_university = df_university.set_index(['State', 'RegionName'])
    ratio_college = df_all_homes.loc[list(df_university.index)]['Ratio'].dropna()
    ratio_not_college_indices = set(df_all_homes.index) - set(ratio_college.index)
    ratio_not_college = df_all_homes.loc[list(ratio_not_college_indices), :]['Ratio'].dropna()
    
    statistic, p_value = tuple(ttest_ind(ratio_college, ratio_not_college))
    
    def better():
        if ratio_college.mean() < ratio_not_college.mean():
            return 'university town'
        else:
            return 'non-university town'
        
    result = statistic < 0
    different = p_value < 0.01
    
    return (different, p_value, better())


run_ttest()